In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications.densenet import DenseNet121 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D 
from tensorflow.keras.models import Model 
from tensorflow.keras import backend as K 
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os, sys
import cv2
import random
%matplotlib inline

In [ ]:
!ls /kaggle/working

In [ ]:
#ROOT_DIR = 
#DATA_SETS = '../datasets'
DATA_SETS = '../input/ranzcr-clip-catheter-line-classification'
#SCRIPTS = 

In [ ]:
datasets_list = os.listdir(DATA_SETS)
datasets_list

In [ ]:
train = os.path.join(DATA_SETS, 'train')
submission = os.path.join(DATA_SETS,'sample_submission.csv')
test = os.path.join(DATA_SETS, 'test')

In [ ]:
train_df = pd.read_csv(DATA_SETS+'/train.csv')
print(f'Train Data CSV: {train_df.shape[0]}')
train_df.head()

In [ ]:
sub_df = pd.read_csv(submission)
print(f'Sub CSV: {sub_df.shape[0]}')
sub_df.head()

In [ ]:
# label column names
label_cols = list(sub_df.columns[1:])
image_labels = train_df[label_cols].values # will be used for train validation splitting

In [ ]:
# Count up the number of instances of each class (drop non-class columns from the counts) 
class_counts = train_df.sum().drop(['StudyInstanceUID','PatientID'])

In [ ]:
# we plot the distribution of patients to check if there is any class imbalance in the dataset
def plot_class_distributions(values, index):
    sns.barplot(x=values, y=index)
    plt.title('Distribution of classes for the patients')
    plt.xlabel('Patient Count', fontsize=15)
    plt.ylabel('Catheter Position', fontsize=15)
    plt.show()

In [ ]:
plot_class_distributions(class_counts.values, class_counts.index)

In [ ]:
train_images = DATA_SETS + "/train/" + train_df['StudyInstanceUID'] + '.jpg'   
test_images = DATA_SETS + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'
len(test_images)

In [ ]:
data_image = os.path.join(DATA_SETS, "train_annotations.csv")
data_image = pd.read_csv(data_image)
data_image.head()

In [ ]:
# display images randomly on a grid
def display_images(image_ids, labels):
    fig = plt.figure(figsize = (16,12))
    
    for index, (image_id, label) in enumerate(zip(image_ids,labels)):
        plt.subplot(3,3, index+1)
        image = image_id + '.jpg'
        image = mpimg.imread(os.path.join(DATA_SETS, "train", image))
        plt.title(label, fontsize=12)
        plt.imshow(image,cmap='Greys')
    
    fig.tight_layout()
    plt.show()    
        

In [ ]:
tmp_train = data_image.sample(9)
image_ids = tmp_train["StudyInstanceUID"].values
labels = tmp_train['label'].values
display_images(image_ids, labels)

In [ ]:
# now we check for any kinds of data leakage between any two data sets, train-test, train-valid, valid-test
def check_for_leakage(df1, df2, patient_col):
    
    df1_unique = set(df1[patient_col])
    df2_unique = set(df2[patient_col])
    
    common_patients = df1_unique.intersection(df2_unique)
    
    return True if len(common_patients) > 0 else False
    
    

In [ ]:
check_for_leakage(train_df, sub_df, 'StudyInstanceUID')

In [ ]:
# for making validation set from given train set
def make_val_set(data, val_size):
    val_patientid = random.sample(list(train_df['PatientID'].unique()),int(val_size*len(train_df['PatientID'].unique())))
    df_train = data[~data['PatientID'].isin(val_patientid)]
    df_val = data[data['PatientID'].isin(val_patientid)]
    return df_train, df_val

In [ ]:
df_train, df_val = make_val_set(train_df, val_size=0.2)

In [ ]:
check_for_leakage(df_train, df_val, 'PatientID')

In [ ]:
print(f'Train Data Size: {df_train.shape[0]}')
print(f'Validation Data Size: {df_val.shape[0]}')

In [ ]:
TRAIN_IMAGE_DIR = DATA_SETS + '/train/'
TEST_IMAGE_DIR = DATA_SETS + '/test/'

In [ ]:
# with augmentation
def get_train_generator(df, image_dir, image_id, label_names, shuffle=True, batch_size=32, seed=1, target_width=256, target_height=256):
    print('Train Generator Preparation: ')
    
    # first we normalize the images and add light augmentation to the images
    image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2
    )
    
    # next we flow from data frame with a certain batch size.
    # This flows the images from the directory
    generator = image_generator.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col=image_id,
        y_col=label_names,
        class_mode="raw",
        classes=label_names,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=1,
        target_size=(target_width, target_height)
    )
    
    return generator
    

In [ ]:
# create image generator
def get_image_generator(train_df, image_dir, image_id, label_names, shuffle=True, batch_size=32, seed=1, target_width=256, target_height=256, sample_size=100):
    # generator to sample dataset
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df,
        directory=image_dir,
        x_col=image_id,
        y_col=label_names,
        class_mode='raw',
        batch_size=sample_size,
        shuffle=shuffle,
        target_size=(target_width, target_height)
    )
    
    batch = raw_train_generator.next() 
    data_sample = batch[0] 
    
    # use sample to fit mean and std for test set generator
    # featurewise mean and normalization is done on all the images
    image_generator = ImageDataGenerator( 
        featurewise_center=True, 
        featurewise_std_normalization= True) 
    
    # fit generator to sample from training data 
    image_generator.fit(data_sample)
    
    return image_generator

In [ ]:
def get_generator(df, image_dir, x_col, y_cols, image_generator, batch_size=32, seed=1, target_width = 256, target_height = 256):
    # get valid generator
    
    generator = image_generator.flow_from_dataframe( 
        dataframe=df, 
        directory=image_dir, 
        x_col=x_col, y_col=y_cols, 
        class_mode="raw", 
        batch_size=batch_size, 
        shuffle=False, 
        seed=seed, 
        target_size=(target_width,target_height)
    )
    
    return generator

In [ ]:
def append_ext(fn, ext='.jpg'):
    return fn+ext

df_train['StudyInstanceUID'] = df_train['StudyInstanceUID'].apply(append_ext)
df_val['StudyInstanceUID'] = df_val['StudyInstanceUID'].apply(append_ext)
sub_df['StudyInstanceUID'] = sub_df['StudyInstanceUID'].apply(append_ext)

In [ ]:
sub_df['StudyInstanceUID'][0]

In [ ]:
train_generator = get_train_generator(df_train, TRAIN_IMAGE_DIR, "StudyInstanceUID", label_cols)
raw_image_generator = get_image_generator(df_train, TRAIN_IMAGE_DIR, "StudyInstanceUID", label_cols)
valid_generator = get_generator(df_val, TRAIN_IMAGE_DIR, "StudyInstanceUID", label_cols, raw_image_generator)
test_generator= get_generator(sub_df, TEST_IMAGE_DIR, "StudyInstanceUID", label_cols, raw_image_generator)

# Model Building

## Custom Weighted Loss Function and Frequency analysis

In [ ]:
test_generator.labels[100]

## Training Model

In [ ]:
!ls /kaggle/checkpoints

In [ ]:
# base pre-trained model
model = DenseNet121(
    include_top=False,
    weights="imagenet",
)

layer = model.output

# adding a global pooling layer
layer = GlobalAveragePooling2D()(layer)

# adding a simple dense layer for testing purposes
dense_layer1 = Dense(len(class_counts.index), activation="sigmoid")(layer)

model = Model(inputs=model.input, outputs=dense_layer1)
checkpoint = ModelCheckpoint('/kaggle/checkpoints/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto') 
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                             validation_data=valid_generator,
                             steps_per_epoch=100,
                             epochs=20,
                   callbacks=[checkpoint])

In [ ]:
def plot_metrices(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()
    
    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
plot_metrices(history)

In [ ]:
predicted_vals = model.predict(test_generator, steps = len(test_generator))

In [ ]:
predicted_vals[0]